In [208]:
import scipy as sp
import numpy as np
import scipy.integrate as spint
import math
import matplotlib.pyplot as plt
import scipy.misc as misc

In [214]:
#Problem one part one
def fa(x):
    return np.exp(np.cos(x))

trua = spint.quad(fa, -np.pi, np.pi)
apts = np.zeros(6)

def fb(x):
    return np.exp(-((x+.1)**2))

trub = spint.quad(fb, -10, 10)
bpts = np.zeros(6)

#part a
for i in range (1, 7):
    pts = np.linspace(-np.pi, np.pi, 2**i)
    fpts = fa(pts)
    apts[i-1] = np.abs(spint.trapz(fpts, x = pts) - trua[0])

print(apts)
#part b
for i in range (1, 7):
    pts = np.linspace(-10, 10, 2**i)
    fpts = fb(pts)
    bpts[i-1] = np.abs(spint.trapz(fpts, x = pts) - trub[0])

print(bpts)

#The first function experiences rapid convergence because it is periodic. Due to the fact that it is periodic, the
#derivatives in the Taylor expansion cancel, leading to the order of the convergence increasing as h shrinks.

#For the second function, it is basically just a typical Gaussian distribution function, which also makes the endpoint
#derivatives negligible. This means that while the derivatives do not fully cancel, they are incredibly small beyond
#what would be expected in normal error analysis, so the function converges faster as h shrinks.

[5.64347182e+00 2.78294112e-01 2.00963690e-05 0.00000000e+00
 8.88178420e-16 0.00000000e+00]
[1.77245385e+00 1.77221107e+00 1.00722024e+00 1.22583993e-02
 1.00354614e-10 6.66133815e-16]


In [210]:
#Problem one part two

def fb(x):
    return np.exp(-((x+.1)**2))*np.exp(-np.abs(x))

def fp(x):
    return misc.derivative(fb, x)

trub = spint.quad(fb, -10, 10)
bpts = np.zeros(6)

for i in range (1, 7):
    pts = np.linspace(-10, 10, 2**i)
    fpts = fb(pts)
    bpts[i-1] = spint.trapz(fpts, x = pts)
    bpts[i-1] = np.abs(bpts[i-1] - trub[0])

#from the points, we can clearly see second order convergence
print(bpts)

#This function has a derivative discontinuity at x = 0, which is in the interval. This means that in the Taylor
#expansion the endpoints trending to 0 does not improve accuracy because of the sharp spike in derivative values near 
#0, which offsets any gain from negligible derivatives near the ends.

[1.08675199 1.08674333 0.90336298 0.20282533 0.03657831 0.00843673]


Problem one part three

Using the error formula, we have that error $E = \frac{-h^2}{12}(f'(b) - f'(a)) + O(h^4)$. Taking the derivative of the function $f(x)$, we know the derivative of $g(x)e^{-|x|} = \frac{e^{-|x|}(|x|g'(x) - xg(x))}{|x|}$. 

We know that $g$ is smooth, so these derivatives exist. We can estimate $g, g'$ by saying that both are equal to $10^{-20}$, since that is given as an upper bound. So then we can plug in these for a and b in the error formula.

We then have $E = \frac{-h^2}{12}(\frac{e^{-|10|}(|10|g'(10) - 10g(10))}{|10|} - \frac{e^{-|-10|}(|-10|g'(-10) + 10g(-10))}{|-10|}) + O(h^4)$.

Simplify...

$E = \frac{-h^2}{12}(\frac{e^{-|10|}(|10|g'(10) - 10g(10) + |10|g'(-10) + 10g(-10))}{|10|} + O(h^4)$ 

$E = \frac{-h^2}{12}(\frac{e^{-|10|}(|10|g'(10) - 10g(10) + |10|g'(-10) + 10g(-10))}{|10|} + O(h^4)$ 

$E = \frac{-h^2}{12}(e^{-10}(g'(10) - g(10) + g'(-10) + g(-10)) + O(h^4)$ 

We know $e^{-10}$ is already on the order of $10^{-5}$, so if in addition $g$ is small, this error function will be approximately $h^2 O(10^{-26}) + O(h^4)$. We cannot get the exact value of C1 since $g$ is not exactly given, but this does give a good upper bound.

Alternately, we can say split the function into two pieces, the positive half from 0 to 10 and the negative half from -10 to 0. Exploring this avenue, we get...

Ignore the $h^2/12$ term for the moment, just for simplicity. Simply consider the derivative component.

The error for the half of the integral from 0 to 10 is $\frac{e^{-|x|}(|x|g'(x) - xg(x))}{|x|}$ at x = 10 minus the same function at x = 0. Unfortunately, the function is undefined at 0, but there may be a solution.

The error for the half of the integral from -10 to 0 is $\frac{e^{-|x|}(|x|g'(x) - xg(x))}{|x|}$ at x = 0 minus the same function at x = -10. This negates the issue of the function being undefined at the point. So now we are just left with $\frac{e^{-|10|}(|10|g'(x) - 10g(10))}{|10|} - \frac{e^{-|-10|}(|-10|g'(-10) + 10g(-10))}{|-10|}$. Of course, this simply returns us to where we were above.


Problem one part four

Our goal is to remove the $h^2$ term. If the function is smooth, we can do this quite simply since it is best approximated by (as per our error formula) by the equation $-\frac{1}{12}(f'(a) - f'(b))h^2$. Simply add this to the trapezoidal formula in each trapezoid to make it fourth order.

In [215]:
#Problem one part five

def fb(x):
    return np.exp(-((x+0.1)**2) - np.abs(x))

def fp(x):
    return misc.derivative(fb, x)

trub = spint.quad(fb, -10, 10, epsabs = 1*10**(-12))

bpts = np.zeros(6)
startval = -10
endval = 10

for i in range (1, 7):
    pts = np.linspace(startval, endval, 2**(2*i))
    fpts = fb(pts)
    fppts = fp(pts)
    corrval = 0
    for k in range(0, len(pts)-1):
        corrval = corrval + (((pts[k+1] - pts[k])**2)/12)*(fppts[k] - fppts[k+1])
    bpts[i-1] = np.abs(spint.trapz(fpts, x = pts) + corrval - trub[0])

#TESTING VERSION, PLEASE IGNORE
#for i in range (1, 7):
#    pts = np.linspace(-10, 10, 2**(i))
#    fpts = fb(pts)
#    fppts = fp(pts)
#    corrval = 0
#    for k in range(0, len(pts)-1):
#        corrval = corrval + (((pts[k+1] - pts[k])**2)/12)*(fppts[k] - fppts[k+1])
#    bpts[i-1] = np.abs(spint.trapz(fpts, x = pts) -corrval - trub[0])

#plt.figure(2)
#plt.loglog(bpts)

print(bpts)

#from the points, we can clearly see approximate fourth order convergence

[1.08674333e+00 2.02825325e-01 8.43673484e-03 5.07967606e-04
 3.15361128e-05 1.96801980e-06]


In [212]:
#Problem two
def f1(x):
    return np.exp(x)*np.cos(x)

def f2(x):
    return np.exp(x**2)/np.sqrt(1-x**2)

def f3(x):
    return np.exp(-x**2)*np.exp(np.cos(x))

def f4(x):
    return np.exp(-x)*np.cos(x)

#part 1
e1 = spint.quad(f1, 0, math.pi)
print("(estimate, error):", e1)
#part 2
e2 = spint.quad(f2, -1, 1)
print("(estimate, error):", e2)
#part 3
e3 = spint.quad(f3, -np.inf, np.inf)
print("(estimate, error):", e3)
#part 4
e4 = spint.quad(f4, 0, np.inf)
print("(estimate, error):", e4)

(estimate, error): (-12.070346316389635, 1.7582219880446364e-13)
(estimate, error): (5.508429773886771, 4.472640391384175e-08)
(estimate, error): (3.9898122725860024, 1.8812109574991397e-08)
(estimate, error): (0.5, 3.1147513347725457e-09)


In [213]:
#Problem three

#This is an adaptive quadrature method in Scipy

f1 = lambda y, x: np.exp(-x**2*1 - y**2*1)
f10 = lambda y, x: np.exp(-x**2*10 - y**2*10)
f100 = lambda y, x: np.exp(-x**2*100 - y**2*100)
#k=1
k1 = spint.dblquad(f1, -1, 1, lambda x: -1, lambda x: 1)
print("(estimate, error):", k1)
#k=10
k10 = spint.dblquad(f10, -1, 1, lambda x: -1, lambda x: 1)
print("(estimate, error):", k10)
#k=100
k100 = spint.dblquad(f100, -1, 1, lambda x: -1, lambda x: 1)
print("(estimate, error):", k100)

(estimate, error): (2.2309851414041346, 2.476891071583481e-14)
(estimate, error): (0.31415439954313085, 8.392849089322585e-11)
(estimate, error): (0.03141592653589761, 1.47119637350431e-08)
